Configurações Iniciais

In [6]:
import findspark

# Inicializa o findspark para encontrar a instalação do Spark
findspark.init()

# Verifica o caminho da instalação do Spark
findspark.find()

# Importa PySpark
import pyspark

# Importa a classe SparkSession do módulo pyspark.sql
from pyspark.sql import SparkSession

# Cria uma nova sessão Spark
spark = SparkSession\
        .builder\
        .appName("Projeto_AJP")\
        
        # Configura o modo de execução local utilizando todos os núcleos disponíveis
        .master("local[*]")\
        
        # Configura a memória do executor Spark (8 GB)
        .config("spark.executor.memory", "8g")\
        
        # Configura a memória do driver Spark (8 GB)
        .config("spark.driver.memory", "8g")\
        
        # Configura o tamanho máximo do resultado do driver (1 GB)
        .config("spark.driver.maxResultSize", "1g")\
        
        # Desabilita o uso de memória off-heap
        .config("spark.memory.offHeap.enabled", False)\  

        # Habilita suporte a Hive
        .enableHiveSupport()\  
        .getOrCreate()


Carregar Dados

In [7]:
# Ler os conjuntos de dados de voos de diferentes anos
flights_2022 = spark.read.csv("../data/raw/Combined_Flights_2022.csv", header=True, inferSchema=True)
flights_2021 = spark.read.csv("../data/raw/Combined_Flights_2021.csv", header=True, inferSchema=True)
flights_2020 = spark.read.csv("../data/raw/Combined_Flights_2020.csv", header=True, inferSchema=True)
flights_2019 = spark.read.csv("../data/raw/Combined_Flights_2019.csv", header=True, inferSchema=True)
flights_2018 = spark.read.csv("../data/raw/Combined_Flights_2018.csv", header=True, inferSchema=True)

# Combinar todos os conjuntos de dados de voos num único DataFrame
all_flights_raw = flights_2022.union(flights_2021).union(flights_2020).union(flights_2019).union(flights_2018)

In [8]:
#Verificar se não se perderam dados

print("Número total de observações em 2018:", flights_2018.count())
print("Número total de observações em 2019:", flights_2019.count())
print("Número total de observações em 2020:", flights_2020.count())
print("Número total de observações em 2021:", flights_2021.count())
print("Número total de observações em 2022:", flights_2022.count())
print("Número total de observações:", all_flights_raw.count())

Número total de observações em 2018: 5689512
Número total de observações em 2019: 8091684
Número total de observações em 2020: 5022397
Número total de observações em 2021: 6311871
Número total de observações em 2022: 4078318
Número total de observações: 29193782


Configuração das Variáveis

In [9]:
#Ver como estão definidas as variaveis do dataset

all_flights_raw.printSchema()
all_flights_raw.show(1, vertical=True)

print("Número de colunas no DataFrame resultante:", len(all_flights_raw.columns))
print("Número total de observações:", all_flights_raw.count())

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

In [10]:
#Seleção de colunas a manter no Dataframe

cols_to_keep = [
    'Year',
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Airline',
    'Origin',
    'Dest',
    'Distance',
    'Cancelled',
    'Diverted',
    'CRSDepTime',
    'DepTime',
    'DepDelayMinutes'
]

In [11]:
all_flights = all_flights_raw.select(cols_to_keep)

In [12]:
all_flights.show(1, vertical=True)

-RECORD 0-------------------------------
 Year            | 2022                 
 Month           | 4                    
 DayofMonth      | 4                    
 DayOfWeek       | 1                    
 Airline         | Commutair Aka Cha... 
 Origin          | GJT                  
 Dest            | DEN                  
 Distance        | 212.0                
 Cancelled       | false                
 Diverted        | false                
 CRSDepTime      | 1133                 
 DepTime         | 1123.0               
 DepDelayMinutes | 0.0                  
only showing top 1 row



In [1]:
#Configurar as variaveis para o tipo pretendido

from pyspark.sql.functions import col

all_flights = all_flights \
    .withColumn("Year", col("Year").cast("int")) \
    .withColumn("Month", col("Month").cast("int")) \
    .withColumn("DayofMonth", col("DayofMonth").cast("int")) \
    .withColumn("DayOfWeek", col("DayOfWeek").cast("int")) \
    .withColumn("Distance", col("Distance").cast("int")) \
    .withColumn("Cancelled", col("Cancelled").cast("boolean")) \
    .withColumn("Diverted", col("Diverted").cast("boolean")) \
    .withColumn("CRSDepTime", col("CRSDepTime").cast("float")) \
    .withColumn("DepTime", col("DepTime").cast("float")) \
    .withColumn("DepDelayMinutes", col("DepDelayMinutes").cast("float"))


In [ ]:
all_flights.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: float (nullable = true)
 |-- DepTime: float (nullable = true)
 |-- DepDelayMinutes: float (nullable = true)



Remoção de Duplicados e Nulos

In [16]:
# Remoção de valores nulos
all_flights_cleaned = all_flights.dropna()

In [17]:
# Remoção de duplicados
all_flights_cleaned = all_flights_cleaned.dropDuplicates()

Criar Sample

In [18]:
sample_flights = all_flights_cleaned.sample(fraction=0.01)

Gravar Dados

In [19]:
# Salvar dados pré-processados
all_flights_cleaned.write.mode("overwrite").parquet("../data/processed/flights_cleaned.parquet")
sample_flights.write.mode("overwrite").parquet("../data/processed/flights_sample.parquet")

Encerrar Spark

In [20]:
spark.stop()